In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import threading

# Load YOLO
net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
classes = []
with open('cfg/coco.names', 'r') as f:
    classes = f.read().splitlines()

# Function to perform object detection on a frame   
def detect_objects(frame):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    if len(indexes) > 0:  # Check if any detections are found
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

    return frame

# Function to open video file
def open_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        cap = cv2.VideoCapture(file_path)
        threading.Thread(target=play_video, args=(cap,)).start()  # Run play_video in a separate thread

# Function to play video
def play_video(cap):
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = detect_objects(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = ImageTk.PhotoImage(image=Image.fromarray(frame))
        label.config(image=frame)
        label.image = frame
        root.update()
    cap.release()

# Function to update GIF
def update_gif():
    global gif_index
    gif_label.config(image=gif_frames[gif_index])
    gif_index = (gif_index + 1) % len(gif_frames)
    root.after(100, update_gif)

# Create Tkinter GUI
root = tk.Tk()
root.title("Object Detection with YOLO")

# Add background image
bg_color1 = "#ffebcd"  # PeachPuff
bg_color2 = "#ffe4e1"  # MistyRose
background = tk.Label(root, bg=bg_color1)
background.place(relwidth=1, relheight=1)

# Load animated GIF frames
gif_frames = []
for i in range(1,13):
    gif_path = f"animated/animated{i}.gif"
    gif = Image.open(gif_path)
    gif = gif.resize((900, 900), Image.ANTIALIAS)
    gif_frames.append(ImageTk.PhotoImage(gif))

gif_index = 0
gif_label = tk.Label(root, bg=bg_color2)
gif_label.place(relx=0.5, rely=0.5, anchor="center")
update_gif()

# Add button with fancy font
btn_open = tk.Button(root, text="Open Video File", command=open_file, bg="#FF69B4", font=("Comic Sans MS", 20), relief=tk.FLAT)
btn_open.pack(pady=10)

# Add label for video display
label = tk.Label(root)
label.pack()

# Make the window resizable
root.resizable(True, True)

# Get the screen width and height
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Set the window size and position
window_width = int(screen_width * 0.8)
window_height = int(screen_height * 0.8)
x_position = int((screen_width - window_width) / 2)
y_position = int((screen_height - window_height) / 2)

# Set the window geometry
root.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")

root.mainloop()

/var/folders/jz/blj7k8ds5sdbmv6xjpzfvdt00000gq/T/ipykernel_1990/176656453.py:103: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  gif = gif.resize((900, 900), Image.ANTIALIAS)
